<a href="https://colab.research.google.com/github/odesay97/MachineLearing_Class/blob/main/5%EC%9E%A5_%EC%8B%A4%EC%8A%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
# 데이터 불러오기 및 전처리

import pandas as pd

dataset = pd.read_csv("https://bit.ly/wine_csv_data")

# dataset.head()
# dataset.info()
# dataset.describe()

import numpy as np

train_data = dataset[['alcohol','sugar','pH']]
target_data = dataset['class']

from sklearn.model_selection import train_test_split, GridSearchCV

train_input, test_input, train_target, test_target = train_test_split(train_data,target_data,stratify=target_data,random_state=46)








결정트리

In [8]:
# 결정트리 최적화
from sklearn.tree import DecisionTreeClassifier

params = { 'max_depth': np.arange(3,20,1),
          'min_impurity_decrease': np.arange(0.0001,0.001,0.0001),
          'min_samples_split': np.arange(2,40,3)

}

gs1 = GridSearchCV(DecisionTreeClassifier(random_state=42),params,n_jobs=1)
gs1.fit(train_input,train_target)

print(gs1.score(train_input,train_target))
print(gs1.score(test_input,test_target))
print(gs1.best_params_)


0.9084564860426929
0.8738461538461538
{'max_depth': 13, 'min_impurity_decrease': 0.00030000000000000003, 'min_samples_split': 8}


In [12]:
# 결정트리 만들고 평가
dc = DecisionTreeClassifier(random_state=42,max_depth=13,min_impurity_decrease=0.0003,min_samples_split=8)
dc.fit(train_input,train_target)

print(dc.score(train_input,train_target))
print(dc.score(test_input,test_target))

0.9084564860426929
0.8738461538461538


In [ ]:
# 랜덤 포레스트

In [ ]:
# 엑스트라 트리

In [ ]:
# 그레디언트 부스팅

In [ ]:
# 히스토그램기반 그래디언트 부스팅